In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = (SparkSession.builder.appName("test_app")
         .master("local[*]")
        .config("spark.sql.warehouse.dir", "/user/hive/warehouse")
        .config("spark.sql.catalogImplementation", "hive")
        .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083")
        .config("spark.hadoop.fs.defaultFS",  "hdfs://namenode:9000")
        .enableHiveSupport()
        .getOrCreate())
spark.sparkContext.setLogLevel("ERROR")

23/08/27 17:28:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
def cells_per_tech_for_all_sites():
    cell_techs = spark.sql("select distinct cell_tech from netwrok_cells.dim__cell_tech").collect()
    site_ids = spark.sql("select distinct site_id from netwrok_cells.site").collect()
    site_and_tech = spark.sql("""
    select count(*) as cell_count, site_id, cell_tech, dt  from netwrok_cells.cell_site
    group by dt, site_id, cell_tech
    order by dt, site_id, cell_tech;
    """)
    return site_and_tech.groupBy("dt").agg(
    *[
        F.max(
            F.when(
                (F.col("cell_tech") == cell_tech.cell_tech)
                & (F.col("site_id") == site_id.site_id),
                F.col("cell_count"),
            ).otherwise(0)
        ).alias(f"{site_id.site_id}_{cell_tech.cell_tech}_cnt")
        for site_id in site_ids
        for cell_tech in cell_techs
    ]
)

In [3]:
cells_per_tech_for_all_sites().show()

+----------+----------+---------+---------+----------+---------+---------+
|        dt|3_umts_cnt|3_lte_cnt|3_gsm_cnt|4_umts_cnt|4_lte_cnt|4_gsm_cnt|
+----------+----------+---------+---------+----------+---------+---------+
|2018-10-24|         2|        4|        2|         0|        0|        0|
|2019-11-04|         0|        0|        2|         0|        0|        0|
+----------+----------+---------+---------+----------+---------+---------+



In [6]:
def cells_per_tech_and_freq_for_site(site_id):
    cell_techs_freqs = spark.sql("select distinct cell_tech, frequency_band from netwrok_cells.dim__cell_tech").collect()
    site_tech_and_freq = spark.sql(f"""
    select count(*) as cell_count, cell_tech, frequency_band, dt  from netwrok_cells.cell_site
    where site_id = '{site_id}'
    group by dt, cell_tech, frequency_band
    order by dt, cell_tech, frequency_band
    ;
    """)
    return site_tech_and_freq.groupBy("dt").agg(
        *[
            F.max(
                F.when(
                    (F.col("cell_tech") == cell_tech_freq.cell_tech)
                    & (F.col("frequency_band") == cell_tech_freq.frequency_band),
                    F.col("cell_count"),
                ).otherwise(0)
            ).alias(f"frequency_band_{cell_tech_freq.cell_tech[0].upper()}{cell_tech_freq.frequency_band}_by_site")
            for cell_tech_freq in cell_techs_freqs
        ]
    )


In [7]:
cells_per_tech_and_freq_for_site(3).show()

+----------+----------------------------+----------------------------+---------------------------+---------------------------+----------------------------+----------------------------+---------------------------+----------------------------+---------------------------+
|        dt|frequency_band_L2600_by_site|frequency_band_L1800_by_site|frequency_band_L700_by_site|frequency_band_G900_by_site|frequency_band_G1800_by_site|frequency_band_U2100_by_site|frequency_band_U900_by_site|frequency_band_L2100_by_site|frequency_band_L800_by_site|
+----------+----------------------------+----------------------------+---------------------------+---------------------------+----------------------------+----------------------------+---------------------------+----------------------------+---------------------------+
|2018-10-24|                           1|                           1|                          1|                          1|                           1|                           2|      